In [4]:
# Add this to the first cell of your test.ipynb

import sys
import os

# Get the absolute path to the root directory (parent of 'notebooks/')
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add root_dir to sys.path if it's not already there
if root_dir not in sys.path:
    sys.path.insert(0, root_dir)

# Optional: Verify that 'src' is now accessible
print("Updated sys.path:")
for path in sys.path:
    print(path)


Updated sys.path:
c:\Users\VPBV528\Downloads\quann\quanvolutional_nn_benchmark
c:\Users\VPBV528\Downloads\quann\quanvolutional_nn_benchmark\src
c:\Users\VPBV528\Downloads\quann\quanvolutional_nn_benchmark\notebooks
c:\Program Files\Python39\python39.zip
c:\Program Files\Python39\DLLs
c:\Program Files\Python39\lib
c:\Program Files\Python39

C:\Users\VPBV528\AppData\Roaming\Python\Python39\site-packages
c:\Program Files\Python39\lib\site-packages
c:\Program Files\Python39\lib\site-packages\win32
c:\Program Files\Python39\lib\site-packages\win32\lib
c:\Program Files\Python39\lib\site-packages\Pythonwin


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import os

from qcml.bench.grid_search import GridSearch
from qcml.utils.log import log_setup

from src.models.quanvolutional_net import QuanvolutionalNet
from src.utils.data_loader import load_mnist_data
from src.circuits.convolution import custom_circuit
from src.circuits.embedding import custom_embedding
from src.circuits.measurement import custom_measurement

# Configure Logging
log_settings = {
    "output": "both",  # Log to both terminal and file
    "terminal_level": "INFO",
    "file_level": "DEBUG",
    "hide_logs": ["jax", "pennylane", "bokeh", "distributed"],
    "slack_notify": False
}

log_setup(**log_settings)


2024-11-01 15:45:02,116 - root - INFO - Logging is set up. (log.py:391)
2024-11-01 15:45:02,130 - root - INFO - jax logs are set to ERROR level. (log.py:421)
2024-11-01 15:45:02,133 - root - INFO - pennylane logs are set to ERROR level. (log.py:421)
2024-11-01 15:45:02,134 - root - INFO - bokeh logs are set to ERROR level. (log.py:421)
2024-11-01 15:45:02,134 - root - INFO - distributed logs are set to ERROR level. (log.py:421)


In [7]:
# Set Environment Variables
os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Enable CuDNN Benchmark for potential speed improvements (optional)
torch.backends.cudnn.benchmark = True


Using device: cpu


In [8]:
X_train, y_train, X_val, y_val = load_mnist_data(batch_size=4, output='np', limit=500)
print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, Validation labels shape: {y_val.shape}")

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Training data shape: (500, 1, 8, 8), Training labels shape: (500,)
Validation data shape: (500, 1, 8, 8), Validation labels shape: (500,)


In [9]:
# Define Hyperparameter Values for Grid Search
qkernel_shapes = [2, 3]  # Quantum kernel shapes to explore
classical_kernel_shapes = [3, 5]  # Classical kernel shapes
embeddings = [custom_embedding]  # List of embedding functions
circuits = [custom_circuit]  # List of circuit functions
measurements = [custom_measurement]  # List of measurement functions

# Example parameter lists (adjust based on your actual parameter needs)
params_list = [
    {'circuit': {'weights': torch.randn(2, 4, 3)}},  # Example parameters for qkernel_shape=2
    {'circuit': {'weights': torch.randn(2, 9, 3)}}   # Example parameters for qkernel_shape=3
]

n_classes = [10]  # Number of output classes
learning_rates = [1e-3, 1e-4]  # Learning rates to explore

# Define the grid
model_grid = { 
    "qkernel_shape": qkernel_shapes,
    "classical_kernel_shape": classical_kernel_shapes,
    "embedding": embeddings,
    "circuit": circuits,
    "measurement": measurements,
    "params": params_list,
    "n_classes": n_classes,
    "learning_rate": learning_rates
}


In [11]:
# Define Experiment Name
embedding_type = "custom_embedding"  # Example embedding type
experiment_name = f"quantum_mnist_{embedding_type}"

# Initialize GridSearch
gs = GridSearch(
    classifiers=[QuanvolutionalNet],
    param_grid=model_grid,
    batch_size=50,  # Batch size for training
    experiment_name=experiment_name,
)

# Run the Grid Search
results = gs.run(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val
)


2024-11-01 16:08:40,800 - qcml.bench.grid_search - INFO - Starting grid search with QuanvolutionalNet, 16 combinations, 8 parallel jobs (grid_search.py:165)


Epoch 1/10, Loss: 2.3041, Accuracy: 0.0960
Epoch 2/10, Loss: 2.3014, Accuracy: 0.1100
Epoch 3/10, Loss: 2.2993, Accuracy: 0.1160
Epoch 4/10, Loss: 2.2977, Accuracy: 0.1240
Epoch 5/10, Loss: 2.2962, Accuracy: 0.1320
Epoch 6/10, Loss: 2.2948, Accuracy: 0.1320
Epoch 7/10, Loss: 2.2931, Accuracy: 0.1320
Epoch 8/10, Loss: 2.2929, Accuracy: 0.1320
Epoch 9/10, Loss: 2.2935, Accuracy: 0.1320
Epoch 10/10, Loss: 2.2918, Accuracy: 0.1320


2024-11-01 16:10:27,732 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 2, 'classical_kernel_shape': 3, 'embedding': <function custom_embedding at 0x0000023C92B1F3A0>, 'circuit': <function custom_circuit at 0x0000023C92B1F040>, 'measurement': <function custom_measurement at 0x0000023C92B1F550>, 'params': {'circuit': {'weights': tensor([[[-0.5794, -1.1714,  0.2140],
         [-0.3145,  0.4082,  0.7496],
         [-0.1207, -0.2265, -1.2132],
         [-1.1280,  0.2981, -0.5385]],

        [[-0.1233, -0.0665,  1.7561],
         [ 0.1177, -0.8981, -0.9276],
         [ 1.6355,  0.9134, -0.9597],
         [-0.4922, -1.0049, -2.8835]]])}}, 'n_classes': 10, 'learning_rate': 0.001}, accuracy: 0.1340 (grid_search.py:241)


Epoch 1/10, Loss: 2.3044, Accuracy: 0.0860
Epoch 2/10, Loss: 2.3048, Accuracy: 0.0840
Epoch 3/10, Loss: 2.3044, Accuracy: 0.0940
Epoch 4/10, Loss: 2.3043, Accuracy: 0.0920
Epoch 5/10, Loss: 2.3037, Accuracy: 0.1020
Epoch 6/10, Loss: 2.3037, Accuracy: 0.0960
Epoch 7/10, Loss: 2.3034, Accuracy: 0.0960
Epoch 8/10, Loss: 2.3036, Accuracy: 0.1020
Epoch 9/10, Loss: 2.3035, Accuracy: 0.1000
Epoch 10/10, Loss: 2.3030, Accuracy: 0.1120


KeyboardInterrupt: 